In [3]:
from urllib import request
import re
import pandas as pd
import urllib.parse as urp
import requests
import json

import csv

In [26]:
def __get_location1__(name,city):  
        my_ak = 'uXuYBYbI90SdKOGSjIqgDnthz6DrkFN4'   
        qurey = urp.quote(name)
        try:
            url = 'http://api.map.baidu.com/place/v2/search?query='+qurey+'&tag='+'&region='+urp.quote(city)+'&output=json&ak='+my_ak
            #print(url)
            req = request.urlopen(url)
            res = req.read().decode()
            temp = json.loads(res)
            lat = pd.to_numeric(re.findall('"lat":(.*)',res)[0].split(',')[0])
            lng = pd.to_numeric(re.findall('"lng":(.*)',res)[0])
            area = re.findall('"area":(.*)',res)[0][:-1]
            return (lng,lat),str(area)  #经度和纬度
        except:
            return (0,0),''

In [11]:
print(__get_location1__('紫华佳苑','上海'))

((121.543922, 31.278916), '"杨浦区"')


### get data

In [15]:
df = pd.read_excel('data_after_clean.xls')
df.head()

,Unnamed: 0,小区,价格,面积,室,厅,卫,楼层,近地铁,随时看房,精装
0,0,山西小区,6700,65.00,2.0,0,NaN,NaN,1.0,NaN,NaN
1,1,明园森林都市滴翠苑,8500,91.00,2.0,2,NaN,NaN,1.0,NaN,NaN
2,2,四元路20弄小区,2600,56.07,2.0,1,NaN,NaN,0.0,NaN,NaN
3,3,明园森林都市滴翠苑,5500,84.00,2.0,2,NaN,NaN,1.0,NaN,NaN
4,4,紫华佳苑,6800,78.00,1.0,2,NaN,NaN,1.0,NaN,NaN


In [36]:
df['Longitude']=0
df['Latitude']=0
df['location']=0
df['stop']=0
df['line']=0
df['distance']=0
df = df[['价格','面积','室','厅','卫','楼层','近地铁','精装','随时看房','小区','Longitude','Latitude','location','stop','line','distance']]
#df = df[['价格','面积','室','厅','卫','楼层','近地铁','精装','随时看房','小区','Longitude','Latitude']]
#df = df[['Price','Area','Longitude','Latitude','Bedrooms','Livingrooms','Bathrooms','Floor','NextToSubway','Exquisite','OpenForVisits','location']]
df.head()

,价格,面积,室,厅,卫,楼层,近地铁,精装,随时看房,小区,Longitude,Latitude,location,stop,line,distance
0,6700,65.00,2.0,0,NaN,NaN,1.0,NaN,NaN,山西小区,0,0,0,0,0,0
1,8500,91.00,2.0,2,NaN,NaN,1.0,NaN,NaN,明园森林都市滴翠苑,0,0,0,0,0,0
2,2600,56.07,2.0,1,NaN,NaN,0.0,NaN,NaN,四元路20弄小区,0,0,0,0,0,0
3,5500,84.00,2.0,2,NaN,NaN,1.0,NaN,NaN,明园森林都市滴翠苑,0,0,0,0,0,0
4,6800,78.00,1.0,2,NaN,NaN,1.0,NaN,NaN,紫华佳苑,0,0,0,0,0,0


### get and label location as dummy variables

In [27]:
address = df['小区']
lng = []
lat = []
location_lst = [] #location 区

for loc in address:
    result = __get_location1__(str(loc),'上海')
    lnglat=result[0] #经纬度    
    location=result[1] #区
    
    while lnglat == (0,0):
        for i in range(10):
            lnglat = __get_location1__(str(loc),'上海')
            if lnglat ==(0,0):
                continue
            else:
                break

    lng.append(str(lnglat[0]))
    lat.append(str(lnglat[1]))
    location_lst.append(location)

ValueError: Length of values does not match length of index

In [37]:
df['location']=location_lst
df['Longitude']=lng
df['Latitude']=lat
dummy = pd.get_dummies(df['location'])
df = pd.concat([df,dummy], axis=1)
df = df.drop(columns="location")
df.shape
df.to_excel('data.xls')

<ipython-input-37-ae5f3be9c476>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location']=location_lst
<ipython-input-37-ae5f3be9c476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude']=lng
<ipython-input-37-ae5f3be9c476>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

### get nearest subway information

In [39]:
def get_subway(lat_partion,lng_partion,radius): 
    my_ak = 'uXuYBYbI90SdKOGSjIqgDnthz6DrkFN4'
    targets=[] 
    lines = [] 
    distances=[] 
    nearstation=[]
    
    for i in range(len(lat_partion)):
        location=str(list(lat_partion)[i])+","+ str(list(lng_partion)[i]) #circle center location (housing)
        print(1,location)
        try:                
            url="http://api.map.baidu.com/place/v2/search?query=交通设施&tag=地铁站&page_size=10&page_num=0&location="+location+"&radius="+str(radius)+"&output=json&ak="+my_ak+"&scope=2"#构造请求网址
            #print(url)
            response=requests.get(url) 
            answer=response.json() 
            
            target=answer['results'][0]['name'] 
            line=answer['results'][0]['address']
            distance=answer['results'][0]['detail_info']['distance']
           
            targets.append(target)
            lines.append(line)
            distances.append(distance)
            nearstation.append(1)
            
        except:
            print("the circle contains no message")
            targets.append("")
            lines.append("")
            distances.append("")
            nearstation.append(0)
            
    return targets,lines,distances,nearstation

In [37]:
df = pd.read_excel('data.xls')
df.head()

,Unnamed: 0,价格,面积,室,厅,卫,楼层,近地铁,精装,随时看房,...,"""秀洲区""","""虎丘区""","""虹口区""","""金山区""","""长宁区""","""闵行区""","""闸北区""","""青浦区""","""静安区""","""黄浦区"""
0,0,6700,65.00,2.0,0,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
1,1,8500,91.00,2.0,2,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
2,2,2600,56.07,2.0,1,NaN,NaN,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,5500,84.00,2.0,2,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
4,4,6800,78.00,1.0,2,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#df = df[1:100]
stop_lst=[]
line_lst=[]
distance_lst=[]
nearstation_lst=[]

lat_partion=df['Latitude']
lng_partion=df['Longitude']
radius=1000

stop_lst,line_lst,distance_lst,nearstation_lst=get_subway(lat_partion,lng_partion,radius)

1 31.240372999999998,121.489484
1 31.295211,121.46085500000001
1 31.419139,121.44033799999998
1 31.295211,121.46085500000001
1 31.278916,121.543922
1 31.629754,121.44136499999999
the circle contains no message
1 31.066803000000004,121.24131200000001
1 31.031397,121.1481
the circle contains no message
1 31.066803000000004,121.24131200000001
1 31.166613,121.420649
1 31.033326000000002,121.294203
the circle contains no message
1 31.218341,121.45958999999999
1 31.260385,121.517277
1 31.072145000000003,121.50143
the circle contains no message
1 31.295211,121.46085500000001
1 31.327516999999997,121.280165
1 31.217384000000003,121.454544
1 31.295211,121.46085500000001
1 31.323794,121.288431
1 31.305653000000003,121.49663600000001
1 31.285515999999998,121.13876599999999
the circle contains no message
1 30.927893,121.444906
the circle contains no message
1 31.046433,121.771693
1 31.067638,121.32174599999999
the circle contains no message
1 31.275391,121.542551
1 31.002406,121.243618
1 31.295211

the circle contains no message
1 31.117246999999995,121.63366699999999
the circle contains no message
1 31.063626,121.758974
1 31.124902000000002,121.508253
1 30.743503999999998,121.36231200000002
the circle contains no message
1 31.270443,121.46118999999999
1 31.225519,121.53497
1 31.290912,121.53155500000001
1 31.362917,121.369832
1 31.327516999999997,121.280165
1 31.299957,121.593025
1 31.308245,121.305401
1 31.248709,121.407554
1 31.12615,121.761559
the circle contains no message
1 31.379676,121.44018899999999
1 31.108354,121.23915600000001
1 31.222778,121.372978
1 31.113417,121.766223
the circle contains no message
1 31.195109999999996,121.63453500000001
1 31.295162,121.373802
the circle contains no message
1 31.211208000000003,121.664001
1 31.224342,121.48529099999999
1 31.106861,121.60803500000002
1 31.234436,121.417273
1 31.181949,121.239466
1 "静安区",(121.459244, 31.235515999999997)
the circle contains no message
1 31.206273,121.370686
1 31.305653000000003,121.49663600000001
1 3

1 31.236187,121.46138
1 31.164274,121.40524599999999
1 31.062464000000002,121.750552
the circle contains no message
1 31.009117,121.23078400000001
1 30.725757,121.36160600000001
the circle contains no message
1 31.424418,121.43114399999999
the circle contains no message
1 31.21068,121.381495
1 31.170696000000003,121.511036
1 30.995599,121.566696
the circle contains no message
1 31.073444,121.762068
the circle contains no message
1 31.256233,121.635038
the circle contains no message
1 31.259493,121.59904399999999
1 31.263675,121.40698300000001
1 31.047848,121.452703
1 31.364615999999998,121.371342
1 31.131572,121.318524
1 31.056338,121.75899
1 31.411513,121.280906
the circle contains no message
1 31.350378999999997,121.353568
the circle contains no message
1 31.216839,121.675777
1 31.195428000000003,121.241752
the circle contains no message
1 31.240648999999998,121.31785900000001
1 31.248717,121.473206
1 31.144732,121.322053
1 31.07222,121.405828
1 31.150083000000002,121.542405
1 31.305

the circle contains no message
1 31.021308,121.246385
1 31.32286,121.453475
1 31.146124,121.325997
1 31.331828,121.27595900000001
1 31.283712,121.362765
1 31.26101,121.38503899999999
the circle contains no message
1 31.187479999999997,121.53402700000001
1 31.261459000000002,121.44743700000001
1 31.250034000000003,121.414524
1 31.060665000000004,121.74360300000001
the circle contains no message
1 31.200996999999997,121.467873
1 31.069002,121.542318
1 31.287748999999998,121.607216
1 nan,(0, 0)
the circle contains no message
1 31.107419,121.254778
the circle contains no message
1 31.045912,121.464941
the circle contains no message
1 31.202978,121.300119
1 31.137116,121.380683
1 30.990534999999998,121.43379099999999
1 31.1469,121.53806599999999
1 31.240594,121.328736
1 31.32286,121.453475
1 31.214496999999998,121.62966499999999
1 nan,(0, 0)
the circle contains no message
1 31.308397999999997,121.46920300000001
the circle contains no message
1 31.143341999999997,121.325851
1 31.249588,121.5

1 31.195351000000002,121.46376599999999
1 31.135032,121.212543
the circle contains no message
1 31.073788,121.40788899999998
1 31.155107,121.551245
1 31.184798999999998,121.52870800000001
1 31.27182,121.59328500000001
1 31.197255,121.466132
1 31.119840999999997,121.386982
1 31.184798999999998,121.52870800000001
1 31.191824,121.645
1 31.266932,121.49371599999999
1 31.318506,121.38136000000002
1 23.134377999999998,113.346401
1 31.205516999999997,121.46834
1 31.194304,121.64053999999999
1 31.290912,121.53155500000001
1 31.103921999999997,121.496197
the circle contains no message
1 31.204853999999997,121.465255
1 31.344409000000002,121.448249
1 31.200243,121.472879
1 31.221375,121.61926299999999
1 31.268340999999996,121.422827
1 31.260761,121.42936399999999
1 31.266932,121.49371599999999
1 30.98034,121.521247
the circle contains no message
1 31.180344,121.36913899999999
1 31.164453,121.36613500000001
1 31.011369000000002,121.432395
1 31.222911,121.31370600000001
the circle contains no mess

...

the circle contains no message
1 31.188603999999998,121.64300700000001
the circle contains no message
1 31.085651000000002,121.517329
the circle contains no message
1 31.151922999999996,121.421885
the circle contains no message
1 31.339209000000004,121.5956
the circle contains no message
1 31.216384,121.63316200000001
the circle contains no message
1 31.216384,121.63316200000001
the circle contains no message
1 31.254348999999998,121.3619
the circle contains no message
1 31.195853000000003,121.435616
the circle contains no message
1 31.226810999999998,121.41977
the circle contains no message
1 31.16491,121.50015900000001
the circle contains no message
1 31.235052000000003,121.407595
the circle contains no message
1 31.220732,121.43089099999999
the circle contains no message
1 31.203234000000002,121.43873500000001
the circle contains no message
1 31.280255,121.56419
the circle contains no message
1 31.152125,121.404344
the circle contains no message
1 31.255418,121.46724099999999
the ci

1 31.216384,121.63316200000001
the circle contains no message
1 31.177684999999997,121.496499
the circle contains no message
1 31.241109,121.427846
the circle contains no message
1 31.182135,121.506955
the circle contains no message
1 31.223872999999998,121.53041599999999
the circle contains no message
1 31.405662,121.444699
the circle contains no message
1 31.279709999999998,121.42828300000001
the circle contains no message
1 31.252928999999998,121.411547
the circle contains no message
1 31.141818,121.40638799999999
the circle contains no message
1 31.355156,121.578666
the circle contains no message
1 31.238327,121.407105
the circle contains no message
1 31.112896999999997,121.24540800000001
the circle contains no message
1 31.263240000000003,121.43484199999999
the circle contains no message
1 31.124316999999998,121.275748
the circle contains no message
1 31.235402,121.42855300000001
the circle contains no message
1 31.351179,121.24698799999999
the circle contains no message
1 31.1948

1 31.180962,121.512257
the circle contains no message
1 31.063502000000003,121.249661
the circle contains no message
1 31.347257,121.45331000000002
the circle contains no message
1 31.204454,121.42623999999999
the circle contains no message
1 31.181315,121.500143
the circle contains no message
1 31.283628000000004,121.51451100000001
the circle contains no message
1 31.289596999999997,121.18581100000002
the circle contains no message
1 31.279232,121.48317
the circle contains no message
1 31.009184,121.428625
the circle contains no message
1 31.220246000000003,121.37291100000002
the circle contains no message
1 31.279632,121.488625
the circle contains no message
1 31.154168,121.431074
the circle contains no message
1 31.229181,121.432197
the circle contains no message
1 31.275873999999998,121.45829099999999
the circle contains no message
1 31.257715,121.49715900000001
the circle contains no message
1 31.201173999999998,121.56251599999999
the circle contains no message
1 31.186356,121.452

the circle contains no message
1 31.280699,121.59125300000001
the circle contains no message
1 31.203528000000002,121.480753
the circle contains no message
1 31.230489000000002,121.42891100000001
the circle contains no message
1 31.231009999999998,121.523067
the circle contains no message
1 31.230728999999997,121.518406
the circle contains no message
1 nan,(0, 0)
the circle contains no message
1 31.250153000000005,121.47167900000001
the circle contains no message
1 nan,(0, 0)
the circle contains no message
1 31.213420000000003,121.42573
the circle contains no message
1 31.166075,121.494165
the circle contains no message
1 31.134862,121.382428
the circle contains no message
1 31.178041999999998,121.544402
the circle contains no message
1 31.238933000000003,121.526397
the circle contains no message
1 31.242931,121.39741599999999
the circle contains no message
1 31.217017,121.40438700000001
the circle contains no message
1 31.190026,121.56939399999999
the circle contains no message
1 31.2

1 31.345914,121.594381
the circle contains no message
1 31.240369,121.530791
the circle contains no message
1 31.247799,121.550021
the circle contains no message
1 31.100004,121.40086299999999
the circle contains no message
1 31.218927,121.52868500000001
the circle contains no message
1 31.289996000000002,121.486833
the circle contains no message
1 31.220378999999998,121.429107
the circle contains no message
1 31.277158,121.588774
the circle contains no message
1 31.23716,121.407614
the circle contains no message
1 31.199537,121.43110300000001
the circle contains no message
1 31.242088,121.41880400000001
the circle contains no message
1 31.296418,121.527454
the circle contains no message
1 31.164745,121.349558
the circle contains no message
1 31.250028000000004,121.47486
the circle contains no message
1 31.294309000000002,121.48238
the circle contains no message
1 31.275402000000003,121.46201599999999
the circle contains no message
1 31.179304,121.49158899999999
the circle contains no 

In [41]:
df['stop']=stop_lst
df['line']=line_lst
df['distance']=distance_lst
df['近地铁']=nearstation_lst

In [42]:
df = df.rename(columns = {'价格': 'Price','面积':'Area', '室':'Bedrooms', '厅':'Livingrooms', '卫':'Bathrooms', '楼层':'Floor', '近地铁':'NextToSubway', '随时看房':'OpenForVisits', '精装':'Exquisite'}, inplace = False)
#df=df.drop(columns=["小区"])
df.to_excel('new_data.xls')